# vDSF heatmap $\omega \, S(|q|, \omega)$ with spherical averaging

This notebook computes and plots a 2D heatmap of the **dynamic structure factor**
$S(|q|, \omega)$ (or its velocity-weighted variant $\omega\,S$), averaged over
directions on the sphere at fixed $|q|$.

**Workflow**
1. Load a **mass-weighted** dynamical matrix $D$ of shape $(3N, 3N)$.
2. Diagonalize: eigenvalues $\omega^2$, eigenvectors are mass-weighted displacements.
3. Build `PhononModel` from your package and compute $S(q, \omega)$ on a shell of
   $|q|$ values with spherical averaging.
4. Plot a publication-ready heatmap and save an NPZ grid.

**Notes**
- If positions are in Angstrom, provide $q$ in 1/Angstrom.
- This notebook assumes your eigenvectors should be treated as
  `eigenvectors_kind="mass_weighted"` and eigenvalues as `eigenvalue_kind="omega_squared"`.
- The vectorized `dynamic_structure_factor(q_vectors=...)` API is used throughout.

In [ ]:
# Parameters (edit here)
from pathlib import Path

# --- Inputs ---
# dynmat_path = Path("data/dynmat.npy")  # (3N, 3N) mass-weighted matrix
positions_path = Path("data/positions.npy")  # optional, shape (N, 3) or None
masses_path = None  # optional, shape (N,)
weights_path = None  # optional, shape (N,)

# --- q shell and averaging ---
qmax = 2.5  # maximum |q|
nq = 60  # number of |q| samples
ndir = 48  # number of directions per |q|
sphere_grid = "fibonacci"  # "fibonacci" or "random"
seed = None  # used only if sphere_grid == "random"

# --- physics / DSF ---
temperature = 300.0
polarization = "longitudinal"  # "longitudinal" or "transverse"
kernel_kind = "gaussian"  # "gaussian" or "lorentzian"
sigma = 0.05  # used for Gaussian; set 0.0 to auto ~ 3 * dω
gamma = 0.00  # used for Lorentzian; set 0.0 to auto ~ 3 * dω

# --- plotting / outputs ---
plot_vdsf = True  # if True, plot omega * S; if False, plot S
normalize_columns = False  # normalize each |q| column by its max
omegamax = 120.0  # optional omega cap for plot and saved grid; set None to skip
vmax = 500.0  # optional colorbar upper bound; set None for auto

pdf_out = Path("sqw_heatmap_notebook.pdf")
npz_out = Path("sqw_grid_notebook.npz")

In [ ]:
# Imports and helpers
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, Literal

from hydroglass.spectra.phonon import Mesh1D, PhononModel


def load_array_any(path: Optional[Path]):
    if path is None:
        return None
    p = path.expanduser().resolve()
    if not p.exists():
        raise FileNotFoundError(f"File not found: {p}")
    if p.suffix == ".npy":
        return np.load(p).astype(float, copy=False)
    if p.suffix == ".npz":
        with np.load(p) as z:
            key = list(z.keys())[0]
            return z[key].astype(float, copy=False)
    if p.suffix == ".txt":
        return np.loadtxt(p).astype(float, copy=False)
    raise ValueError(f"Unsupported file extension: {p.suffix}")


def build_omega_mesh(
    frequencies: np.ndarray,
    overshoot: float = 0.2,
    points: int = 3001,
    omegamax: Optional[float] = None,
) -> Mesh1D:
    fmin = max(0.0, float(np.min(frequencies)))
    fmax = float(np.max(frequencies))
    span = max(fmax - fmin, 1.0)
    lo = max(0.0, fmin - overshoot * span)
    hi = fmax + overshoot * span
    if omegamax is not None and np.isfinite(omegamax) and omegamax > 0.0:
        hi = min(hi, float(omegamax))
    return Mesh1D(points=np.linspace(lo, hi, int(points)))


def fibonacci_sphere(n: int) -> np.ndarray:
    if n <= 0:
        raise ValueError("ndir must be positive.")
    ga = np.pi * (3.0 - np.sqrt(5.0))
    k = np.arange(n, dtype=float)
    z = 1.0 - 2.0 * (k + 0.5) / n
    r = np.sqrt(np.maximum(0.0, 1.0 - z * z))
    theta = ga * k
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    dirs = np.stack([x, y, z], axis=1)
    norms = np.linalg.norm(dirs, axis=1, keepdims=True)
    return (dirs / np.maximum(norms, 1e-15)).astype(float)


def random_sphere(n: int, seed: Optional[int]) -> np.ndarray:
    if n <= 0:
        raise ValueError("ndir must be positive.")
    rng = np.random.default_rng(seed)
    xyz = rng.normal(size=(n, 3))
    norms = np.linalg.norm(xyz, axis=1, keepdims=True)
    return (xyz / np.maximum(norms, 1e-15)).astype(float)


def build_q_vectors_spherical_average(
    q_abs: np.ndarray,
    ndir: int,
    scheme: Literal["fibonacci", "random"],
    seed: Optional[int],
) -> np.ndarray:
    if scheme == "fibonacci":
        dirs = fibonacci_sphere(ndir)
    else:
        dirs = random_sphere(ndir, seed=seed)
    q_all = q_abs[:, None, None] * dirs[None, :, :]
    return q_all.reshape(q_abs.size * ndir, 3).astype(float)

## Load and diagonalize the mass-weighted dynamical matrix

We interpret eigenvalues as $\omega^2$ and eigenvectors as mass-weighted
displacements. Tiny negative eigenvalues from numerical noise are clipped to zero.

In [ ]:
# D = load_array_any(dynmat_path)
# assert D.ndim == 2 and D.shape[0] == D.shape[1], "dynmat must be square"
# ndof = D.shape[0]
# assert ndof % 3 == 0, "dynmat dimension must be multiple of 3"
# natoms = ndof // 3

positions = load_array_any(positions_path) if positions_path is not None else None
masses = load_array_any(masses_path) if masses_path is not None else None
weights = load_array_any(weights_path) if weights_path is not None else None

natoms = (
    positions.shape[0]
    if positions is not None
    else (
        masses.shape[0]
        if masses is not None
        else (weights.shape[0] if weights is not None else None)
    )
)

if positions is not None:
    assert positions.shape == (natoms, 3)
if masses is not None:
    assert masses.shape == (natoms,)
if weights is not None:
    assert weights.shape == (natoms,)

# Dense diagonalization
# w2, V = np.linalg.eigh(D)
# w2 = np.clip(w2, a_min=0.0, a_max=None)
# order = np.argsort(w2)
# w2 = w2[order]
# V = V[:, order]

w2 = np.load("data/eigval.npy")  # (3N,) omega^2
V = np.load("data/eigvec.npy")  # (3N, 3N) mass-weighted
ndof = w2.shape[0]

omega_for_mesh = np.sqrt(np.clip(w2, 0.0, None))
omega_mesh = build_omega_mesh(
    omega_for_mesh, overshoot=0.2, points=3001, omegamax=omegamax
)
domega = float(np.mean(np.diff(omega_mesh.points)))

model = PhononModel(
    frequencies=w2,
    eigenvectors=V,
    masses=masses,
    positions=positions,
    scattering_weights=weights,
    q_vectors=None,
    omega_mesh=omega_mesh,
    eigenvectors_kind="mass_weighted",
    eigenvalue_kind="omega_squared",
)
omega_mesh.points[:5], omega_mesh.points[-5:], domega

## Build $|q|$ shell and directions; compute $S(q, \omega)$ in vectorized form

We optionally plot $\omega\,S$ (vDSF). You can normalize each column for visibility.

In [ ]:
q_abs = np.linspace(0.0, float(qmax), int(nq))
q_vectors = build_q_vectors_spherical_average(q_abs, int(ndir), sphere_grid, seed)
nq_total = q_vectors.shape[0]
assert nq_total == int(nq) * int(ndir)

if kernel_kind == "lorentzian":
    gam = float(gamma) if gamma and gamma > 0.0 else max(3.0 * domega, 1e-6)
    broadening = {"kind": "lorentzian", "gamma": gam}
else:
    sig = float(sigma) if sigma and sigma > 0.0 else max(3.0 * domega, 1e-6)
    broadening = {"kind": "gaussian", "sigma": sig}

omega, sqw_many = model.dynamic_structure_factor(
    q_vectors=q_vectors,
    temperature=float(temperature),
    polarization=str(polarization),
    broadening=broadening,
    chunk_n_modes=None,
)

if plot_vdsf:
    sqw_many = omega[:, None] * sqw_many

if omegamax is not None and np.isfinite(omegamax) and omegamax > 0.0:
    keep = omega <= float(omegamax)
    omega = omega[keep]
    sqw_many = sqw_many[keep, :]

sqw_grid = sqw_many.reshape(omega.size, int(nq), int(ndir)).mean(axis=2)

if normalize_columns:
    col_max = np.maximum(np.max(sqw_grid, axis=0), 1e-16)
    sqw_grid = sqw_grid / col_max[None, :]

omega.shape, sqw_grid.shape

## Plot heatmap and save the grid

The color scale shows either $S(q, \omega)$ or $\omega\,S(q, \omega)$ if the toggle
is enabled above.

In [ ]:
fig, ax = plt.subplots(figsize=(7.6, 4.6), constrained_layout=True)
extent = [q_abs.min(), q_abs.max(), float(omega.min()), float(omega.max())]
im = ax.imshow(
    sqw_grid,
    origin="lower",
    aspect="auto",
    extent=extent,
    interpolation="nearest",
    cmap="RdYlBu_r",
    vmin=0,
    vmax=(float(vmax) if vmax is not None else None),
)
ax.set_xlabel("$|q|$")
ax.set_ylabel("$\omega$")
title_style = "$\omega \,S$" if plot_vdsf else "S"
ax.set_title(f"{title_style}$(|q|, \omega)$  ({polarization}, T={temperature})")
cbar = fig.colorbar(im, ax=ax, pad=0.02)
cbar.set_label("$\omega S(q, \omega)$" if plot_vdsf else "S(q, omega)")

In [ ]:
npz_out.parent.mkdir(parents=True, exist_ok=True)
np.savez_compressed(npz_out, q_abs=q_abs, omega=omega, sqw_grid=sqw_grid)
pdf_out.parent.mkdir(parents=True, exist_ok=True)
fig.savefig(pdf_out, dpi=300)
print(f"Saved grid to {npz_out.resolve()}")
print(f"Saved figure to {pdf_out.resolve()}")

## Quick column check (inspect a single |q| cut)

This helps validate peak positions and qualitative features before using the
full heatmap.

In [ ]:
j = int(0.25 * (nq - 1))  # pick a quarter of the |q| range
fig2, ax2 = plt.subplots(figsize=(6.0, 3.5), constrained_layout=True)
ax2.plot(omega, sqw_grid[:, j], lw=1.5)
ax2.set_xlabel("omega")
ax2.set_ylabel("omega*S" if plot_vdsf else "S")
ax2.set_title(f"Column at |q| = {q_abs[j]:.3f}")